In [ ]:
#data preprocessing 
import pandas as pd
import os 

path = os.getcwd()
filename = "ml-20m/ratings.csv"
full_path = os.path.join(path, filename)

movie_data = pd.read_csv(full_path,delimiter= ",")


NUM_DATASET=2000
#sum(food_data.groupby('userId').count()['movieId']>100)
movie_data_100 = movie_data.groupby('userId').agg(['count'])

movie_user_id = movie_data_100[(movie_data_100.movieId>100)['count']].sample(NUM_DATASET,random_state=12)


selected_movie_data=  movie_data[movie_data['userId'].isin(movie_user_id.index)]

id_list = list(movie_title_data.movieId)
result_mt = []
for user in selected_movie_data.userId.unique():
    temp = selected_movie_data[selected_movie_data['userId'].isin([user])]
    temp_movie_list = list(temp.movieId)
    result_mt.append([temp_movie_list.count(i) for i in id_list])
    
final_movie_2000 = pd.DataFrame(result_mt,index=selected_movie_data.userId.unique(),columns =movie_title_data.title )
final_movie_2000.to_csv('final_movie_2000.csv',sep=',')

In [ ]:
#parameter tuning for CF
from LDA_Gibbs_Sampling import LDA
import pandas as pd
import os 
from sklearn.model_selection import train_test_split
import time 
import numpy as np

start_time= time.time()

path = os.getcwd()
filename = "final_movie_2000.csv"
full_path = os.path.join(path, filename)

movie_data = pd.read_csv(full_path,sep= ',',index_col=False)
movie_data = movie_data.drop(movie_data.columns[0],axis=1)
#movie_data.head()

train, test= train_test_split(movie_data, test_size = 0.1,random_state=3)
train_train, train_test= train_test_split(train, test_size = 0.2,random_state=3)

start_time =time.time()
perplexity_list = []
for i in [5,15,25,30,45, 60,100]:
    lda = LDA(num_topic=i)
    lda.fit(train_train.values)
    ##randomly
    predict_use_data = train_test.copy()
    predict_index = list()
    
    for num,i in enumerate(train_test.values):
        #print(np.nonzero(i)
        nonzero_index = i.nonzero()[0]
        index = np.random.randint(0, len(nonzero_index),dtype=np.intc)
        final_index = nonzero_index[index]
        predict_index.append(final_index)
        predict_use_data.iloc[num,final_index]= 0
    
    lda.predict(predict_use_data.values)
    perplexity = 0
    for num, i in enumerate(predict_index):
        perplexity += np.log(lda.results["predict_term_distribution"][num, i])
    perplexity = np.exp(-perplexity/(num+1))
    perplexity_list.append(perplexity)
    present_time=time.time()
    print("This iteration --- {} seconds ---".format(round(present_time - start_time,2)))
    start_time = present_time


In [ ]:
#simple plot of perplexity
import matplotlib.pyplot as plt
plt.plot(perplexity_list)
plt.show()

In [ ]:
## Evaluation for LDA methods
from LDA_Gibbs_Sampling import LDA
import pandas as pd
import os 
from sklearn.model_selection import train_test_split
import time 
import numpy as np

start_time= time.time()

path = os.getcwd()
filename = "final_movie_2000.csv"
full_path = os.path.join(path, filename)

movie_data = pd.read_csv(full_path,sep= ',',index_col=False)
movie_data = movie_data.drop(movie_data.columns[0],axis=1)
movie_data.head()

train, test= train_test_split(movie_data, test_size = 0.1,random_state=3)
#train_train, train_test= train_test_split(train, test_size = 0.2,random_state=)

start_time =time.time()
perplexity_list = []
for i in [15]:
    lda = LDA(num_topic=i)
    lda.fit(train.values)
    ##randomly
    predict_use_data = test.copy()
    predict_index = list()
    size_list = list()
    
    for num,i in enumerate(test.values):
        #print(np.nonzero(i)
        nonzero_index = i.nonzero()[0]
        size = round(len(i.nonzero()[0])/5)
        size_list.append(size) 
        #index = np.random.randit(0, len(nonzero_index),dtype=np.intc)
        #final_index = nonzero_index[index]
        #predict_index.append(final_index)
        #predict_use_data.iloc[num,final_index]= 0
        index =np.random.choice(nonzero_index,replace=False,size=size)
        predict_index.append(index)
        predict_use_data.iloc[nun,index]= 0
        
    lda.predict(predict_use_data.values)

In [ ]:
#Recall rate computation
recall= list()
test_user = len(size_list)
for top_n in range(0,1000):
    recall_rate = 0
    for i in range(test_user):
        #top_n= 5000
        top_n_index = np.argpartition(lda.results['predict_term_distribution'][i],-top_n)[-top_n:]
        like_list = set(predict_index[i]).intersection(top_n_index)
        recall_rate += len(like_list)/len(predict_index[i])
    recall.append(recall_rate/test_user)
print(recall)